In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import ast

In [3]:
print(pd.__version__)

2.0.2


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('data/detailed_matches.csv')

In [6]:
df.shape

(26099, 5)

In [7]:
df.head(10)

,match_id,start_time,team0_id,team1_id,data
0,7194183433,1686401915,8831040.0,8533458.0,{'result': {'players': [{'account_id': 9170670...
1,7194159264,1686400969,8864178.0,9018528.0,{'result': {'players': [{'account_id': 1470752...
2,7194108785,1686399318,8975678.0,8838314.0,{'result': {'players': [{'account_id': 1402517...
3,7194103293,1686398614,9018528.0,8864178.0,{'result': {'players': [{'account_id': 1525266...
4,7194092128,1686398325,8687717.0,8790282.0,{'result': {'players': [{'account_id': 3550429...
5,7194076021,1686397444,8533458.0,8831040.0,{'result': {'players': [{'account_id': 1545290...
6,7194068615,1686397072,8970359.0,8970358.0,{'result': {'players': [{'account_id': 2362143...
7,7194041424,1686395814,8893840.0,8893837.0,{'result': {'players': [{'account_id': 1425359...
8,7194026082,1686395338,8790282.0,8687717.0,{'result': {'players': [{'account_id': 1734807...
9,7194014163,1686394529,8829924.0,8794067.0,{'result': {'players': [{'account_id': 3127228...


In [8]:
df.isna().sum()

match_id        0
start_time      0
team0_id      262
team1_id      251
data            0
dtype: int64

In [9]:
df = df.dropna()

In [10]:
df.shape

(25689, 5)

In [11]:
df['data'] = df['data'].apply(ast.literal_eval)

In [12]:
df['dt'] = df["data"].apply(lambda x: x.get("result", {}).get("start_time"))
df['dt'] = pd.to_datetime(df['dt'], unit='s')

In [13]:
df['league_id'] = df["data"].apply(lambda x: x.get("result", {}).get("league_id"))

In [14]:
df['year'] = df['dt'].dt.year
df['month'] = df['dt'].dt.month
df['day'] = df['dt'].dt.day

df = df.drop(columns = ['dt', 'match_id'])

In [15]:
all_hero_ids = set(range(1, 141))

for hero_id in all_hero_ids:
    for team_number in range(2):
        column_name = f'hero_{hero_id}_team_{team_number}'
        df[column_name] = df['data'].apply(lambda x: 1 if any(player.get('hero_id') == hero_id \
                                 and player.get('team_number') == team_number for player in x.get('result', {}).get('players', [])) else 0)

In [16]:
df['result'] = df['data'].apply(lambda x: 1 if x.get('result', {}).get('radiant_win') else 0)
df["duration"] = df["data"].apply(lambda x: x.get("result", {}).get("duration"))
df["score"] = df["data"].apply(lambda x: x.get("result", {}).get("radiant_score") +  x.get("result", {}).get("dire_score"))

In [17]:
df.groupby(['result'])['result'].count()

result
0    13375
1    12314
Name: result, dtype: int64

In [18]:
df.head(10)

,start_time,team0_id,team1_id,data,league_id,year,month,day,hero_1_team_0,hero_1_team_1,...,hero_137_team_1,hero_138_team_0,hero_138_team_1,hero_139_team_0,hero_139_team_1,hero_140_team_0,hero_140_team_1,result,duration,score
0,1686401915,8831040.0,8533458.0,{'result': {'players': [{'account_id': 9170670...,None,2023,6,10,0,0,...,0,0,0,0,0,0,0,1,1119,34
1,1686400969,8864178.0,9018528.0,{'result': {'players': [{'account_id': 1470752...,None,2023,6,10,0,0,...,0,0,0,0,0,0,0,1,2404,100
2,1686399318,8975678.0,8838314.0,{'result': {'players': [{'account_id': 1402517...,None,2023,6,10,0,0,...,0,0,1,0,0,0,0,0,1294,36
3,1686398614,9018528.0,8864178.0,{'result': {'players': [{'account_id': 1525266...,None,2023,6,10,0,0,...,0,0,0,0,0,0,0,0,1784,79
4,1686398325,8687717.0,8790282.0,{'result': {'players': [{'account_id': 3550429...,None,2023,6,10,0,0,...,0,0,0,0,0,0,0,1,1942,36
5,1686397444,8533458.0,8831040.0,{'result': {'players': [{'account_id': 1545290...,None,2023,6,10,0,0,...,0,0,0,0,0,0,0,1,2311,71
6,1686397072,8970359.0,8970358.0,{'result': {'players': [{'account_id': 2362143...,None,2023,6,10,0,0,...,0,0,0,0,0,0,0,0,3012,88
7,1686395814,8893840.0,8893837.0,{'result': {'players': [{'account_id': 1425359...,None,2023,6,10,0,0,...,0,0,1,0,0,0,0,0,1931,77
8,1686395338,8790282.0,8687717.0,{'result': {'players': [{'account_id': 1734807...,None,2023,6,10,0,0,...,0,0,0,0,0,0,0,0,1356,40
9,1686394529,8829924.0,8794067.0,{'result': {'players': [{'account_id': 3127228...,None,2023,6,10,0,0,...,0,0,0,0,0,0,0,1,1943,60


In [19]:
df = df.drop(columns = ['data'])

In [69]:
df.to_csv('data/heroes.csv', index=False)